In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
from os.path import normpath,join,dirname
sys.path.append(normpath(join(dirname(__file__), '..')))
from torch import nn
import torch
from IncrementalFewShotModel.layers import framework as framework
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

In [ ]:
class ProtoNet(nn.Module):
    def __init__(self):
        super(ProtoNet, self).__init__()

    # def prototype_generator(self, support, N, K, hidden_size):
    #     support = support.view(-1, K, hidden_size)
    #     prototypes = torch.mean(support, 1).view(-1, N, hidden_size)
    #     return prototypes

    def mergePrototypes(self, both_query, prototypes, totalQ, hidden_size):
        both_query = both_query.view(-1, totalQ, hidden_size).unsqueeze(2)
        prototypes = prototypes.unsqueeze(1)
        dists = torch.pow(both_query - prototypes, 2).sum(-1)
        weights = torch.softmax(-dists, -1)
        prototypes = prototypes.expand(-1, totalQ, -1, -1)
        weights = weights.unsqueeze(3)
        merge_prototypes = torch.sum(prototypes * weights, 2)
        return merge_prototypes

    def merge_protoNet(self, query1, query2, prototypes1, prototypes2, totalQ, hidden_size):
        merged_representation = []
        for i in range(query1.size(0)):  # iterate over each frame
            frame_query1 = query1[i].unsqueeze(0)
            frame_query2 = query2[i].unsqueeze(0)
            meta_merge_prototypes = self.mergePrototypes(frame_query1, prototypes1, totalQ, hidden_size)
            base_merge_prototypes = self.mergePrototypes(frame_query2, prototypes2, totalQ, hidden_size)
            meta_weight = torch.pow(frame_query1 - meta_merge_prototypes.view(-1, hidden_size), 2).sum(-1).unsqueeze(1)
            base_weight = torch.pow(frame_query2 - base_merge_prototypes.view(-1, hidden_size), 2).sum(-1).unsqueeze(1)
            merge_weight = torch.softmax(torch.cat((-base_weight, -meta_weight), 1), -1).unsqueeze(2)
            merge_query = torch.cat((frame_query2.unsqueeze(1), frame_query1.unsqueeze(1)), 1)
            frame_merged_representation = torch.sum(merge_query * merge_weight, 1)
            merged_representation.append(frame_merged_representation)
        return torch.stack(merged_representation)  # stack the list into a tensor


In [ ]:
# Assume transformer1 and transformer2 are your transformer instances
output_transformer1 = transformer1(input_data)
output_transformer2 = transformer2(input_data)

# Assume you have functions to generate prototypes
prototypes1 = generate_prototypes(support_set1)
prototypes2 = generate_prototypes(support_set2)

# Assume merge_protoNet_instance is an instance of the ProtoNet class
merged_representation = merge_protoNet_instance.merge_protoNet(output_transformer1, output_transformer2, prototypes1, prototypes2, totalQ, hidden_size)
